In [11]:
import ollama
import pandas as pd


In [12]:
# Loading the file
file_path = '/Users/antonidziwura/Desktop/Warwick 25/EC349/EC349 Project/processed_data.csv'
df = pd.read_csv(file_path)

In [22]:
def classify_with_llama(name):
    # Check if the name contains indicators of a couple
    if " and " in name.lower() or " & " in name.lower():
        return "3"  # Return 3 for couples
    
    prompt = f"""
    You will not write any words. You will only return a single number:
    1 for Female names
    2 for Male names
    3 for Couples or multiple people
    4 for Unsure or cannot determine
    For example, if the name is 'Sarah', return: 1
    If the name is 'John', return: 2
    If the name is 'John and Sarah' or 'John & Sarah', return: 3
    If the name is ambiguous, unclear, or not a typical personal name, return: 4
    Classify the following name:
    {name}
    """

    # Loading the model
    model_name = "llama3.2:latest"

    # Prompting the model
    response = ollama.chat(model=model_name, messages=[
        {"role": "user", "content": f"{prompt}"}
    ])
    
    # Clean the response - take only the first character that is a digit
    response_text = response["message"]["content"].strip()
    for char in response_text:
        if char.isdigit() and char in ['1', '2', '3', '4']:
            return char
    
    # If no valid digit found, return 4 (unsure)
    return "4"

# ... existing code ...

In [23]:
# Define the run for the n rows
df_test = df.head(43763).copy()

# Run the function
df_test['Host_gender'] = df_test['host_name'].apply(
    lambda x: int(classify_with_llama(x))
)
